In [1]:
import time
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests as rq
import re
from time import sleep
from random import randint
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from IPython.core.display import clear_output


##From the previous python script: BurppleCafeLinks [Depending on how you name it]!!!!!!! - I named other location differently
##By using the excel, it saves time, so that we won't need to reload the whole burpple tp/raffles cafes and extract their links
##And also to standardise how many links there are
##because as time goes, there may be more burpple tp/raffles cafes added and the data will not match with the previous data scrapped


xl_workbook = pd.ExcelFile('C:/Users/Mei Xuan/Downloads/CompiledCafeLinks.xlsx')  # Load the excel workbook
##change accordingly to where you downloaded the excel sheet
##just go properties and see where you download but remb to change \ to /
##python do not read like our computer location eg: C:\Users\Mei Xuan\...

df = xl_workbook.parse("Data")  # Parse the sheet into a dataframe #" xx " the name of sheet 1
urllist = df['Links'].tolist()
        

price = []
food = []
review = []
cafename = []
image = []

for c_url in urllist:
    
# Tell Selenium to get the URL you're interested in.
    sleep(randint(2,3))
    #Print out which url python is currently processing
    driver = webdriver.Chrome()
    driver.get('https://www.burpple.com' + c_url)
    print("Processing Individual Cafes' URL: {}".format('https://www.burpple.com' + c_url))
    clear_output(wait = True)
    
     #you can use clear_output, everytime new print out is out. So that the python field will not be flooded with url links
     ###from IPython.core.display import clear_output
     #clear_output(wait = True)
     #you can choose not to if you want to indivdually check to see where is the error(which url cause it) if any
     #i didnt use it because i want to see how to process works
    
    soup = bs(driver.page_source,"lxml")
    
    ##only extract data if there is even a must order category in the cafe's burpple webpage
    
    if (soup.find("div",{"class": "collection-feed collection-feed--dishes"})) is not None:
        
        container1 = soup.find("div",{"class": "collection-feed collection-feed--dishes"})
        cafe_cont = container1.find_all("div",{"class":"food food--dish card feed-item"})
        name_cont = soup.find("div",{"class": "col-lg-9"})
        
    

        y = len(cafe_cont)
    
        #name of cafe
        cn1 = name_cont.find("h1").text.strip('\n\r\t": ')
    
       #how many times to repeat the same name of the cafe by counting the no. of reviews
        for x in range(0,y):
            it = cn1
            cafename.append(it)



        for item in cafe_cont:
        
        
        ##the try and except Attribute Error cannot lump them up together
        ##Because need to try them out individually to see if it exist and if not, you put a '-' 
        
            try:
                price_elem = item.find("div",{"class": "food-price"}).text
                price_strip = price_elem.strip('\n\$": ')
                floatprice = float(price_strip)
                
            except AttributeError:
                floatprice = '-'
                
            try:
                rev_elem = item.find("div",{"class": "food-description-body"}).text
                rev_strip = rev_elem.strip('Reviews": ')
                intrev = int(rev_strip)
                
            except AttributeError:
                intrev = '-'
                
            
            food_elem = item.find("div",{"class": "food-description-title"}).text
            urlimg = item.find("div",{"class":"food-image"}).get('style')
            imgtext = urlimg.strip("/background-image: url(''); : ")

            print(food_elem)
            clear_output(wait = True)
            
            image.append(imgtext)
            price.append(floatprice)
            food.append(food_elem)
            review.append(intrev)
            
    


Processing Individual Cafes' URL: https://www.burpple.com/starbucks-133?bp_ref=%2Fsearch%2Fsg


In [2]:
mustorder = pd.DataFrame({
        "Cafe_Name": cafename,
        "Item_Name" : food,
        "Item_No_Of_Reviews" : review,
        "Item_Price" : price,
        "Image" : image
    
    })

mustorder

,Cafe_Name,Image,Item_Name,Item_No_Of_Reviews,Item_Price
0,Grain Traders (Telok Ayer),https://burpple-2.imgix.net/foods/177651b19dcf...,White Cold Brew,4,-
1,Grain Traders (Telok Ayer),https://burpple-3.imgix.net/foods/5c1981e39c66...,Mawashi,4,17.5
2,Parallel,https://burpple-2.imgix.net/foods/37f4dcca6285...,Superstar Bowl,4,8.9
3,Kyū Coffee Bar,https://burpple.imgix.net/foods/28311768c563fb...,Drip Coffee,1,4.5
4,JAR'D by Sarnies,https://burpple-3.imgix.net/foods/35ba339963a1...,Tuna Tataki Salad,4,12.5
5,Kraftwich by Swissbake (Raffles Place),https://burpple.imgix.net/foods/3e9eda60f8fec9...,Smoked Salmon Sandwich,4,8.9
6,Kraftwich by Swissbake (Raffles Place),https://burpple-3.imgix.net/foods/2e8c85a79423...,Bread and Butter Pudding,1,6.9


In [4]:
ex = pd.ExcelWriter('CompiledMustOrder.xlsx', options={'encoding':'utf-8'})
mustorder.to_excel(ex, 'Data', index=False)